In [1]:
import pandas as pd

## Conceptual model:

# Data Staging: 
## Extract, Transform and Load (ETL)

<img src = "4142DIAGRAM.png" style ="width: 800px">

Extracting data

In [2]:
df= pd.read_csv('./healthcare-dataset-stroke-data.csv')

Updating column names to to ensure uniformity 

In [3]:
df = df.rename({'Residence_type' : 'residence_type'},axis = 'columns')
df = df.replace( {'formerly smoked':'Formerly smoked', 'never smoked': 'Never smoked', 'smokes':'Smokes', 'children': 'Children'})

Dropping missing values

In [4]:
df = df.dropna()

Ensuring missing values have been dropped

In [5]:
df.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

## Binning Data 

Starting with binning age group

In [6]:
df['age_group'] = pd.cut(df['age'], bins=[0,10,20,30,40,50,60,70,80,90,100], right=False, labels= ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90+'])

Binning BMI

In [7]:
df['bmi_category'] = pd.cut(df['bmi'],bins=[0,18.5,25,30,40,100], right=False, labels= ['Underweight','Normal weight', 'Obese class I', 'Obese class II', 'Morbidly obese'])

Binning glucose

In [8]:
df['glucose_range'] = pd.cut(df['avg_glucose_level'],bins=[0,200,500], right=False, labels= ['Non-diabetic','Diabetic'])

Binning smoking status

In [9]:
#df['ever_smoked'] = df['smoking_status']

df['ever_smoked'] = df['smoking_status'].replace( {'Formerly smoked':'Yes', 'Never smoked': 'No', 'Smokes':'Yes', 'Children': 'No'})

Binning number of risk factors each person has (glucose_range, smoked, bmi, hypertension, heart disease)

In [10]:
def riskChecker(x):
    counter = 0 
    tmpHypertension = x.hypertension
    tmpHeartDisease = x.heart_disease
    tmpBMI = x.bmi_category
    tmpGlucose = x.glucose_range
    tmpSmoked = x.ever_smoked
    if tmpHypertension == 1:
        counter += 1
    if tmpHeartDisease == 1: 
        counter += 1
    if tmpBMI == 'Obese class I' or tmpBMI == 'Obese class II' or tmpBMI ==  'Morbidly obese': 
        counter += 1
    if tmpGlucose == 'Diabetic':
        counter += 1 
    if tmpSmoked == x.ever_smoked == 'Yes':
        counter += 1
    
    return counter


In [11]:
df['risk_factor_count'] = df.apply(riskChecker, axis = 1)

Generating Surrogate Keys

In [12]:
df['surr_key'] = range(1,len(df)+1)

# Export to csv cleaned data

In [23]:
df.to_csv('./clean_stroke_data.csv')

# Creating and Inputting into DB

In [15]:
fact_table_dataframe = pd.DataFrame(df,
                  columns = ['surr_key','demo_key','health_key','personal_key','age','bmi','risk_factor_count','stroke'])
personal_attribute_table = pd.DataFrame(df,
                  columns = ['surr_key', 'gender','age'])
demographic_info_table = pd.DataFrame(df,
                  columns = ['surr_key','ever_married', 'work_type','residence_type'])
health_attribute_table = pd.DataFrame(df,
                  columns = ['surr_key','hypertension', 'heart_disease','smoking_status',
                             'avg_glucose_level','bmi'])                             

      surr_key  demo_key  health_key  personal_key   age   bmi  \
0            1       NaN         NaN           NaN  67.0  36.6   
2            2       NaN         NaN           NaN  80.0  32.5   
3            3       NaN         NaN           NaN  49.0  34.4   
4            4       NaN         NaN           NaN  79.0  24.0   
5            5       NaN         NaN           NaN  81.0  29.0   
...        ...       ...         ...           ...   ...   ...   
5104      4905       NaN         NaN           NaN  13.0  18.6   
5106      4906       NaN         NaN           NaN  81.0  40.0   
5107      4907       NaN         NaN           NaN  35.0  30.6   
5108      4908       NaN         NaN           NaN  51.0  25.6   
5109      4909       NaN         NaN           NaN  44.0  26.2   

      risk_factor_count  stroke  
0                     4       1  
2                     2       1  
3                     2       1  
4                     1       1  
5                     2       1  
...

In [26]:
fact_table_dataframe['demo_key'] = fact_table_dataframe['surr_key']
fact_table_dataframe['health_key'] = fact_table_dataframe['surr_key']
fact_table_dataframe['personal_key'] = fact_table_dataframe['surr_key']
fact_table_dataframe = fact_table_dataframe.drop('surr_key', axis=1)

personal_attribute_table = personal_attribute_table.rename({'surr_key' : 'personal_key'},axis = 'columns')
demographic_info_table = demographic_info_table.rename({'surr_key' : 'demo_key'},axis = 'columns')
health_attribute_table = health_attribute_table.rename({'surr_key' : 'health_key'},axis = 'columns')

fact_table_cols = ",".join([str(i) for i in fact_table_dataframe.columns.tolist()])
personal_attribute_cols = ",".join([str(i) for i in personal_attribute_table.columns.tolist()])
demographic_info_cols = ",".join([str(i) for i in demographic_info_table.columns.tolist()])
health_attribute_cols = ",".join([str(i) for i in health_attribute_table.columns.tolist()])

KeyError: 'surr_key'

In [27]:
from configparser import ConfigParser
import psycopg2
from typing import Dict


def load_connection_info(
    ini_filename: str
) -> Dict[str, str]:
    parser = ConfigParser()
    parser.read(ini_filename)
    # Create a dictionary of the variables stored under the "postgresql" section of the .ini
    conn_info = {param[0]: param[1] for param in parser.items("postgresql")}
    return conn_info


def create_db(
    conn_info: Dict[str, str],
) -> None:
    # Connect just to PostgreSQL with the user loaded from the .ini file
    psql_connection_string = f"user={conn_info['user']} password={conn_info['password']}"
    conn = psycopg2.connect(psql_connection_string)
    cur = conn.cursor()

    # "CREATE DATABASE" requires automatic commits
    conn.autocommit = True
    sql_query = f"CREATE DATABASE {conn_info['database']}"

    try:
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        cur.close()
    else:
        # Revert autocommit settings
        conn.autocommit = False


def create_table(
    sql_query: str, 
    conn: psycopg2.extensions.connection, 
    cur: psycopg2.extensions.cursor
) -> None:
    try:
        # Execute the table creation query
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        conn.rollback()
        cur.close()
    else:
        # To take effect, changes need be committed to the database
        conn.commit()

if __name__ == "__main__":
    # host, database, user, password
    conn_info = load_connection_info("db.ini")

    # Create the desired database
    create_db(conn_info)

    # Connect to the database created
    connection = psycopg2.connect(**conn_info)
    cursor = connection.cursor()

    # Create the "demographic_info" table
    demographic_info = """
        CREATE TABLE demographic_info (
            demo_key INTEGER PRIMARY KEY,
            ever_married TEXT,
            work_type TEXT,
            residence_type TEXT
        )
    """
    create_table(demographic_info, connection, cursor)



    # Create the "health_attribute" table
    health_attribute = """
        CREATE TABLE health_attribute (
            health_key INTEGER PRIMARY KEY,
            hypertension INTEGER,
            heart_disease INTEGER,
            smoking_status TEXT,
            avg_glucose_level DOUBLE PRECISION,
            bmi DOUBLE PRECISION
        )
    """
    create_table(health_attribute, connection, cursor)

    # Create the "personal_attributes" table
    personal_attributes  = """
        CREATE TABLE personal_attributes (
            personal_key INTEGER PRIMARY KEY,
            gender TEXT,
            age DOUBLE PRECISION
        )
    """
    create_table(personal_attributes, connection, cursor)

    # Create the "fact_table" table
    fact_table = """
        CREATE TABLE fact_table (
            demo_key INTEGER REFERENCES demographic_info(demo_key),
            health_key INTEGER REFERENCES health_attribute(health_key),
            personal_key INTEGER REFERENCES personal_attributes(personal_key),
            age DOUBLE PRECISION,
            bmi DOUBLE PRECISION,
            risk_factor_count INTEGER,
            stroke INTEGER 
        )
    """
    create_table(fact_table, connection, cursor)


    
    
    

    

    # Adding personal attribute table
    for i,row in personal_attribute_table.iterrows():
        personal_attr_sql = "INSERT INTO personal_attributes (personal_key, gender, age) VALUES (%s, %s, %s)"
        cursor.execute(personal_attr_sql, tuple(row))
        connection.commit()
        
    # Adding demographic info table
    for i,row in demographic_info_table.iterrows():
        demographic_info_sql = "INSERT INTO demographic_info (demo_key,ever_married,work_type,residence_type) VALUES (%s, %s, %s,%s)"
        cursor.execute(demographic_info_sql, tuple(row))
        connection.commit()

    # Adding health attribute table
    for i,row in health_attribute_table.iterrows():
        health_attribute_sql = "INSERT INTO health_attribute (health_key,hypertension,heart_disease,smoking_status,avg_glucose_level,bmi) VALUES (%s, %s, %s,%s,%s,%s)"
        cursor.execute(health_attribute_sql, tuple(row))
        connection.commit()

    # Adding fact table
    for i,row in fact_table_dataframe.iterrows():
        fact_table_sql = "INSERT INTO fact_table (demo_key, health_key , personal_key, age, bmi, risk_factor_count, stroke) VALUES (%s, %s, %s,%s, %s, %s, %s)"
        cursor.execute(fact_table_sql, tuple(row))
        connection.commit()
        
    connection.close()
    cursor.close()


DuplicateDatabase: database "stroke_db" already exists

Query: b'CREATE DATABASE stroke_db'
DuplicateTable: relation "demographic_info" already exists

Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_married TEXT,\n            work_type TEXT,\n            residence_type TEXT\n        )\n    '
InterfaceError: cursor already closed
Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_married TEXT,\n            work_type TEXT,\n            residence_type TEXT\n        )\n    '
InterfaceError: cursor already closed
Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_married TEXT,\n            work_type TEXT,\n            residence_type TEXT\n        )\n    '
InterfaceError: cursor already closed
Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_marr

InterfaceError: cursor already closed